# B92 Project
ALIKHANI, LUCE, VANHESTE (Project 5)

# Context


First, we have to understand what **symetric encryption** is.

Symmetric encryption is a type of encryption where only one key (a secret key) is used to both encrypt and decrypt electronic information. 

If Alice wants to transmit a document to Bob, she will encrypt the file with the secret key and transmit the cypher to Bob.

Then, Bob will decrypt it using the same secret key.

![Texte alternatif…](https://i.ibb.co/LNqrSNG/20200603-181810.jpg)

It is widely used for the encryption of large amounts of data because it's far most efficient than asymemetric encryption.

Typical symetric encryption algorithms are AES and 3DES.

The entities communicating via symmetric encryption must exchange the key so that it can be used in the decryption process.
This is where asymemetric encryption is used. We won't explain what it is, because it's not necessary.

However, the asymetric encryption security is **computational based**, that is to say based on problems that are too long to solve for classic computers.

In the context of quantum computers, these asymetric systems will become obsolete and new key distribution systems have to be designed. We will see one of those : the B92 protocol.



# B92 protocol : explanation

## B92 in a nutshell

In the protocol, it is assumed that Alice and Bob can communicate over a quantum channel and an unsecured channel.

First, Alice will randomly choose N bits. She will encode each of them with a qubit :
- a 0 is a horizontal state
- a 1 is a diagonal state 

![Texte alternatif…](https://i.ibb.co/N6YWvBF/20200603-194429.jpg)

Then, she will transmit these qubits to Bob via the quantum channel.

To measure these qubits, Bob will have to randomly choose one of these 2 bases of measurement :
- the rectilinear basis
- the diagonal basis

![Texte alternatif…](https://i.ibb.co/T8tCjHk/20200603-194400.jpg)


Let's note all the possible outcomes :

<img src="https://i.ibb.co/DD12FjC/aaaa.png" width="800">


Depending of the measurement, we can make the following conclusions :

- if it's a minus state :<br>
this corresponds to a 0 bit for sure
- if it's a vertical state :<br>
this corresponds to a 1 bit for sure
- if it's a horizontal state : <br>
this is an **ambiguous** case : this measure could come either from a 0 or a 1 bit. Thus, we can't deduce anything about it, and it will be discarded.
- if it's a plus state : <br>
this is an **ambiguous** case : this measure could come either from a 0 or a 1 bit. Thus, we can't deduce anything about it, and it will be discarded.


Then, the logic is for Bob to use the states that surely corresponds to a bit.
He will discard the unsure bits in the key, and send to Alice over the insecure channel the indexes of the bits he discarded.

Then, Alice will discard the same bits and both will have the same key.

![Texte alternatif…](https://i.ibb.co/hK8ZsbG/20200603-194328.jpg)

## Verification step

Technically, at this step we already have the key.

However, the protocol specify an additional step to detect eavesdroppers. In this step, Alice and Bob share a subset of the key (which will be of course discarded). The goal is here to compare them and measure the percentage of differences.

If this percentage is under a known level (which depends of the noise on the quantum channel and the error rates of the measurement tools), then we will use the key.

However, if it's above this level, we know for sure that there is an eavesdropper. Let's see why, take the situation where Eve can intercept on the quantum channel. 

![Texte alternatif…](https://i.ibb.co/X5mXcsG/20200603-211246.jpg)

We know that when Eve measures a quantum state (she is forced to do so because she cannot clone an unknown state), the state will match the result of the measurement. However, we have seen that some states Alice can send have different measure outcomes in some basis. Hence, Eve will introduce errors in the Bob generating key process.

We will test that in the simulation.

# Simulation

##Transmit one bit

In [ ]:
import numpy as np
from collections import deque
import hashlib
from typing import Iterable, Deque, Tuple

np.random.seed(0)

Let's define all the basis and states we will use (visual reminder in the theorical part)

In [ ]:
rectilinear_basis = np.array([[1, 0], 
                              [0, 1]])
diagonal_basis = np.array([[1, -1], 
                           [1, 1]]) / np.sqrt(2)

state_H = np.array([0, 1])
state_plus = np.array([1, 1] / np.sqrt(2))
state_V = np.array([1, 0])
state_minus = np.array([1, -1] / np.sqrt(2))

In [ ]:
def probabilities(psi : np.ndarray, basis : np.ndarray) -> np.ndarray:
  """Get the probability to obtain each element of the base

  :param psi: current state
  :type psi: np.ndarray
  :param basis: measurement basis
  :type basis: np.ndarray
  :return: array of probabilities
  :rtype: np.ndarray
  """
  
  return np.abs(np.conj(basis)@psi)**2

### Alice part

Alice will select a random bit,encode it into a state and send it.

In [ ]:
bit = np.random.randint(2)
print(f"Bit is {bit}")

Bit is 0


In [ ]:
def bit_to_polarized_photon(bit : int) -> np.ndarray:
  """Convert a "computer" bit to a state

  :param bit: the bit to convert
  :type bit: int
  :return: state
  :rtype: np.ndarray
  """
  
  if bit==0:
    return state_H
  if bit==1:
    return state_plus

In [ ]:
state_to_send = bit_to_polarized_photon(bit)
print(f"State sent is {state_to_send}")

State sent is [0 1]


###Bob part

Bob will choose a random basis among the rectilinear and the diagonal ones, and measure the state sent by Alice

In [ ]:
received_state = state_to_send

def get_random_measurement_basis() -> np.ndarray:
  """Get one of the two measurement basis (rectilinear or diagonal)

  :return: base
  :rtype: np.ndarray
  """
  
  return([rectilinear_basis, diagonal_basis][np.random.randint(2)])

measurement_basis = get_random_measurement_basis()
print(f"The measurement basis of Bob is {measurement_basis}")

The measurement basis of Bob is [[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]


In [ ]:
probabilities(received_state, measurement_basis)

array([0.5, 0.5])

In [ ]:
measure = measurement_basis[np.random.choice(len(measurement_basis), 1, p=probabilities(received_state, measurement_basis))]
measure

array([[0.70710678, 0.70710678]])

In [ ]:
def get_bit_from_measurement(measure : np.ndarray) -> int:
  """Give the bit that corresponds to the measure done by Bob
  WARNING : -1 mean the measure is not conclusive
  
  :param measure: state we measured
  :type measure: np.ndarray
  :return: bit
  :rtype: int
  """

  if np.allclose(measure, state_H):
    return -1
  if np.allclose(measure, state_V):
    return 1
  if np.allclose(measure, state_plus):
    return -1
  if np.allclose(measure, state_minus):
    return 0

get_bit_from_measurement(measure)

-1

## Exchange the whole key

Let's implement all that. 

For the quantum channel, I implemented it with a deque because it could allow us to implement more functionalities to the quantum channel.

In [ ]:
def Alice_send(quantum_channel : Deque, sent_array : Iterable) -> None:
  """Simulate Alice generating a random bit and sending its corresponding state into the quantum channel
  Sent_array is for debug purpose, storing what bits and states are sent

  :param quantum_channel: quantum channel, implemented with a double ended queue
  :type quantum_channel: Deque
  :param sent_array: stocking all [bit, state]
  :type sent_array: Iterable
  """

  bit = np.random.randint(2)
  state_to_send = bit_to_polarized_photon(bit)
  quantum_channel.append(state_to_send)
  sent_array.append([bit, state_to_send])

In [ ]:
def Bob_receive(quantum_channel : Deque, received_array : Iterable) -> None:
  """Simulate Bob measuring the state sent by alice on the quantum channel and associating the corresponding bit (-1 if not possible)

  :param quantum_channel: quantum channel, implemented with a double ended queue
  :type quantum_channel: Deque
  :param received_array: stocking all [state, measure, bit]
  :type received_array: Iterable
  """
  
  received_state = quantum_channel.pop()
  measurement_basis = get_random_measurement_basis()
  measure = measurement_basis[np.random.choice(len(measurement_basis), 1, p=probabilities(received_state, measurement_basis))]
  received_array.append([received_state, measurement_basis, get_bit_from_measurement(measure)])

In [ ]:
def Bob_synchronise(received_array : Iterable) -> Tuple[str, np.ndarray]:
  """Simulate Bob getting the indexes of the discarded bits, return the final key and the array of indexes to send to Alice to synchronise the keys

  :param received_array: to get the bits he (Bob) deduced
  :type received_array: Iterable
  :return: the final key and the array of discarded indexes
  :rtype: Tuple[str, np.ndarray]
  """

  received_array = np.array(received_array)
  indexes = np.where(received_array[:,2]!=-1)[0]
  key = ''.join(map(str, received_array[indexes,2]))
  to_synchro = np.zeros(received_array.shape[0])
  to_synchro[indexes] = 1
  return key, to_synchro

In [ ]:
def Alice_synchronise(synchro_token : np.ndarray, sent_array: np.ndarray) -> str:
  """Simulate Alice synchronising the key with Bob by deleting the same indexes  

  :param synchro_token: indexes given by Bob
  :type synchro_token: np.ndarray
  :param sent_array: to get the original bits she (Alice) sent
  :type sent_array: np.ndarray
  :return: the final key and the array of discarded indexes
  :rtype: str
  """

  sent_array = np.array(sent_array)
  indexes = np.where(synchro_token==1)[0]
  key = ''.join(map(str, sent_array[indexes,0]))
  return key


In [ ]:
def compare_key_subsets(key1 : str, key2 : str, n_bits : int) -> float:
  """Utility function to take a random subset of both keys (same indexes) and compare them  

  :param key1: first string
  :type key1: str
  :param key2: second string
  :type key2: str
  :param n_bits: number of characters to compare
  :type n_bits: int
  :return: percentage of errors
  :rtype: float
  """

  indexes = np.random.choice(len(key1), n_bits, replace=False)
  key1 = np.array(list(key1))[indexes]
  key2 = np.array(list(key2))[indexes]
  nb_errors = sum(key1[i] != key2[i] for i in range(n_bits))
  return 100 * nb_errors / float(n_bits)


In [ ]:
quantum_channel = deque()
sent_array=[]
received_array=[]

for _ in range(1024):
  Alice_send(quantum_channel, sent_array)
  Bob_receive(quantum_channel, received_array)

key1, token = Bob_synchronise(received_array)
key2 = Alice_synchronise(token, sent_array)
sub_error = compare_key_subsets(key1, key2, len(key1)//3)

print(f"Hash key1 : {hashlib.sha224(key1.encode()).hexdigest()}\nHash key2 : {hashlib.sha224(key2.encode()).hexdigest()}\nErrors during key generation : {100*len(key1)/len(sent_array)} %")
print(f"Errors in key subset comparaison (1/3 of the key) : {sub_error}%")

Hash key1 : 1f738b54355e8769169f42e611e1b1e5ef0f77c04e1f364b451f6a97
Hash key2 : 1f738b54355e8769169f42e611e1b1e5ef0f77c04e1f364b451f6a97
Errors during key generation : 26.46484375 %
Errors in key subset comparaison (1/3 of the key) : 0.0%


The keys generated by both parts are identical. We did it !

We can notice that during the key generation process, ~25% of the bits are discarded.

##Eve eavesdropping

Let's introduce Eve. She will make a measure the state in the quantum channel before Bob. We will see how the detection during the last step will handle that.

In [ ]:
def Eve_measurement(quantum_channel : Deque) -> None:
  """Simulate Eve eavesdropping on the quantum channel by measuring with a random base (like Bob)

  :param quantum_channel: quantum channel, implemented with a double ended queue
  :type quantum_channel: Deque
  """

  intercept_state = quantum_channel.pop()
  measurement_basis = get_random_measurement_basis()
  measure = measurement_basis[np.random.choice(len(measurement_basis), 1, p=probabilities(intercept_state, measurement_basis))][0]
  quantum_channel.append(measure)


In [ ]:
quantum_channel = deque()
sent_array=[]
received_array=[]

for _ in range(2048):
  Alice_send(quantum_channel, sent_array)
  Eve_measurement(quantum_channel)
  Bob_receive(quantum_channel, received_array)

key1, token = Bob_synchronise(received_array)
key2 = Alice_synchronise(token, sent_array)
sub_error = compare_key_subsets(key1, key2, len(key1)//3)

print(f"Hash key1 : {hashlib.sha224(key1.encode()).hexdigest()}\nHash key2 : {hashlib.sha224(key2.encode()).hexdigest()}\nErrors during key generation : {100*len(key1)/len(sent_array)} %")
print(f"Errors in key subset comparaison (1/3 of the key) : {sub_error}%")

Hash key1 : 9174c1874b0ec968b1abff841dd55d7e75f34815a4b586a4ca373ddb
Hash key2 : e4b343e03f21e9deafc680ee64f175108a68e1e5a93e1440d77cc86d
Errors during key generation : 37.060546875 %
Errors in key subset comparaison (1/3 of the key) : 32.015810276679844%


Differences are detected, Alice and Bob can infer they are listened.

## Noisy quantum channel

Let's introduce a basic noise into the quantum channel and analyse the error rate.

In [ ]:
def noise_introduction(quantum_channel : Deque) -> None:
  """Simulate a noise on the quantum channel (kind of a gaussian noise)
  TODO: contact the "Qubit noise channels" team ?
  
  :param quantum_channel: quantum channel, implemented with a double ended queue
  :type quantum_channel: Deque
  """

  intercept_state = quantum_channel.pop().astype(np.float64)
  intercept_state += np.random.normal(0,0.1,2)
  intercept_state = intercept_state / np.linalg.norm(intercept_state)
  quantum_channel.append(intercept_state)


In [ ]:
quantum_channel = deque()
sent_array=[]
received_array=[]

for _ in range(2048):
  Alice_send(quantum_channel, sent_array)
  noise_introduction(quantum_channel)
  Bob_receive(quantum_channel, received_array)

key1, token = Bob_synchronise(received_array)
key2 = Alice_synchronise(token, sent_array)
sub_error = compare_key_subsets(key1, key2, len(key1)//3)

print(f"Hash key1 : {hashlib.sha224(key1.encode()).hexdigest()}\nHash key2 : {hashlib.sha224(key2.encode()).hexdigest()}\nErrors during key generation : {100*len(key1)/len(sent_array)} %")
print(f"Errors in key subset comparaison (1/3 of the key) : {sub_error}%")

Hash key1 : b1a6284d9a8f0eca2fc2365703d74f6791ba5f03b972329b6453ac0d
Hash key2 : 83ee479b9973203d64edb980d50e38b6d19dba94d74d50d49f6ba3ed
Errors during key generation : 25.341796875 %
Errors in key subset comparaison (1/3 of the key) : 2.3121387283236996%


The error between the keys is smaller than when Eve was eavesdropping. Thus in theory, we can make a distinction between the two cases.

# To go further 

BB92 goals is to achieve a **key distribution**.
During a secure conversation, key distribution algorithm is only used once at the beginning of the conversation to distribute the key.
His goal is to achieve authentification : to ensure that the person that claims to be Alice or Bob is actually Alice or Bob. Indeed, good cryptography  between A and B is not enough : one can ensure that A and B are actually A and B.


The key distribution algorithm is **time consuming** and is not used for every communication during the discussion. Indeed, it will be impossibe to stream quickly if we apply a negociation scheme as B92 before sending each packet.
Consequently, once the key have been distributed, A and B communicate thanks to **symmetric algorithm** (the key of those algorithm is encrypted by the quantum distributed key). 


The **ANSSI** (French National Agence in Charge of Cybersecurity) produced a repport on QKD to assess if they should be used or not.
They tackle the issue concerning the threat of quantum computing on cryptography. It is a very intersting paper both politicaly and scientifically. There is a link in the biography.

Here a few statements : 

* "Quantum Key Distribution (QKD) is a functional equivalent of the usual asymmetric key negotiation mechanisms used in most secure communication protocols deployed on the Internet or in private networks. The particularity of QKD would be that **it has a higher level of security that would justify its deployment for high security applications**. However, the **inherent constraints** of this technology do not allow for its massive deployment that would offer high security guarantees in practice. Moreover, even taking into account the possible emergence of new threats to current cryptological means, foremost among which is the universal quantum computer, the future of secure communications can be assured over time **without the need to use QKD**. Thus, while this technology may eventually be called upon to play a role in niche applications, **it is not the natural evolutionary path for secure communications**."

* "this position paper does not concern quantum communications in general, nor networks for transporting quantum states. These topics are largely prospective, in contrast to QKD."

* " The point here is not to pronounce on the advisability of investing in research and innovation in the fields of QKD or quantum communications, which have porosity with other subjects such as quantum computing. Such investments are likely to be breakthroughs; they are essential to support European scientific and technical excellence and contribute to the development of its strategic autonomy."

* "The security guarantees provided in principle by the QKD are provided **at the cost of heavy employment constraints** which reduce the scope of the services offered and compromise the level of security that can be achieved in practice, particularly in scenarios where communications pass through a network of interconnected QKD links. While the use of QKD on point-to-point links can nevertheless be envisaged as a complementary measure to conventional cryptographic means in a defence-in-depth logic, the expenditure involved in such a choice should not be at the expense of the fight against current threats to information systems. "



# Bibliography

https://www.st-andrews.ac.uk/physics/quvis/simulations_html5/sims/cryptography-b92/B92_photons.html

http://www.rri.res.in/quic/qkdactivities.php

https://www.ssi.gouv.fr/en/publication/should-quantum-key-distribution-be-used-for-secure-communications/?fbclid=IwAR1sBFkKKVkq2-dL3O1Hn6PRzI6Y-GYM87bZ5rHeCuqT8P_4KumTsm2Aips